<a href="https://colab.research.google.com/github/shibakyrc0123-arch/A-Gps-Versiones/blob/main/Ver_1_3_%7C_A_Gps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from datetime import timedelta
import time

# --- 1. CONFIGURACIÓN INICIAL ---

# Geocerca Bodega Monte Carlo (Tus coordenadas exactas)
bodega_coords = [
    (4.101740989185771, -73.65614987790829),
    (4.10275779142336, -73.65779357282884),
    (4.103576674506101, -73.65722519691684),
    (4.103303969059117, -73.65525092470286)
]
bodega_polygon = Polygon(bodega_coords)

# Geolocalizador
geolocator = Nominatim(user_agent="tracker_final_v10")

def obtener_direccion(lat, lon):
    try:
        time.sleep(1) # Pausa para respetar a Nominatim
        location = geolocator.reverse((lat, lon), exactly_one=True)
        # Simplificamos la dirección para que no sea tan larga en el reporte
        return location.address.split(',')[0] if location else "Ubicación desconocida"
    except:
        return "Error de mapa"

def encontrar_archivo():
    rutas = ['/content/', '/content/sample_data/']
    print("🔍 Buscando archivo...")
    for r in rutas:
        if os.path.exists(r):
            for f in os.listdir(r):
                if (f.endswith('.xls') or f.endswith('.xlsx')) and "california" not in f:
                    return os.path.join(r, f)
    return None

# --- 2. EL ALGORITMO LÓGICO ---

def generar_reporte_narrativo():
    archivo = encontrar_archivo()
    if not archivo:
        return "❌ No encontré el archivo en sample_data."

    print(f"📂 Procesando: {archivo}")

    # Lectura robusta
    try:
        df = pd.read_excel(archivo)
    except:
        try:
            df = pd.read_csv(archivo)
        except:
            return "❌ Error de formato de archivo."

    # Normalización
    df.columns = df.columns.str.upper().str.strip()
    if 'FHSERVER' in df.columns:
        df = df.drop(columns=['FHSERVER'])

    # Filtro Año y Orden
    df['FHEVENTO'] = pd.to_datetime(df['FHEVENTO'])
    df = df.sort_values(by='FHEVENTO').reset_index(drop=True)

    if df.empty: return "Archivo vacío."
    anio = df['FHEVENTO'].iloc[0].year
    df = df[df['FHEVENTO'].dt.year == anio]

    # --- VARIABLES DE ESTADO ---
    reporte = []

    # Control de Encendido (Solo el primero)
    ya_reporto_encendido = False

    # Control de Bodega
    dentro_bodega = False

    # Control de Estacionamiento (Agrupación compleja)
    estacionado_activo = False
    est_inicio = None
    est_fin = None
    est_coords = None # (lat, lon)

    # Control Ciudad (Entrada/Salida)
    ultimo_evento_ciudad = None

    print("⏳ Analizando eventos y coordenadas (esto toma un momento)...")

    # Iteramos fila por fila
    total_filas = len(df)

    for i, row in df.iterrows():
        # Progreso visual
        if i % 20 == 0: print(f"Procesando fila {i} de {total_filas}...", end='\r')

        fecha = row['FHEVENTO']
        hora_fmt = fecha.strftime('%I:%M %p').lower().replace("am", "a.m.").replace("pm", "p.m.")

        raw_evento = str(row['EVENTO']).upper().strip()

        # Parsear Coordenadas
        try:
            coords_str = str(row['COORDS']).replace(" ", "")
            lat, lon = map(float, coords_str.split(','))
            punto_actual = Point(lat, lon)
            coords_tuple = (lat, lon)
        except:
            continue # Si no hay coords, saltamos

        # -----------------------------------------------------
        # LÓGICA 1: BODEGA MONTE CARLO (Evento Virtual)
        # -----------------------------------------------------
        esta_dentro_ahora = bodega_polygon.contains(punto_actual)

        if esta_dentro_ahora and not dentro_bodega:
            reporte.append(f"-{hora_fmt} Entrada Bodega Monte Carlo")
            dentro_bodega = True
        elif not esta_dentro_ahora and dentro_bodega:
            reporte.append(f"-{hora_fmt} Salida Bodega Monte Carlo")
            dentro_bodega = False

        # -----------------------------------------------------
        # LÓGICA 2: ESTACIONAMIENTO (Agrupación Inteligente)
        # -----------------------------------------------------
        # Definimos si esta fila cuenta como "Estar quieto"
        es_senal_quieto = False

        # Si el evento explícito es estacionado
        if "ESTACIONADO" in raw_evento:
            es_senal_quieto = True
        # O si ya estamos estacionados, y la distancia es muy corta (<50m), ignoramos si dice "Encendido/Apagado"
        elif estacionado_activo:
            distancia = geodesic(est_coords, coords_tuple).meters
            if distancia < 60: # Margen de tolerancia GPS
                es_senal_quieto = True

        # MAQUINA DE ESTADOS DE ESTACIONAMIENTO
        if es_senal_quieto:
            if not estacionado_activo:
                # INICIO DE PARADA
                estacionado_activo = True
                est_inicio = fecha
                est_coords = coords_tuple

            # ACTUALIZAMOS EL FINAL (Extendemos la parada)
            est_fin = fecha

        else:
            # SE MOVIÓ O CAMBIÓ DE ESTADO (Fuera del rango)
            if estacionado_activo:
                # CERRAMOS EL CICLO ANTERIOR
                duracion_min = (est_fin - est_inicio).total_seconds() / 60

                # Solo reportamos si paró más de 2 minutos (para evitar semáforos)
                if duracion_min > 2:
                    # Obtenemos dirección SOLO al final para ahorrar tiempo
                    if dentro_bodega:
                         # Si las coordenadas caen en la bodega, forzamos el nombre
                         dir_str = "Bodega Monte Carlo"
                    else:
                         dir_str = obtener_direccion(*est_coords)

                    h_ini = est_inicio.strftime('%I:%M')
                    h_fin = est_fin.strftime('%I:%M %p').lower().replace("am", "a.m.").replace("pm", "p.m.")

                    # Formato solicitado: "-8:00 / 8:27 a.m. Se estaciona en..."
                    reporte.append(f"-{h_ini} / {h_fin} Vehículo estacionado en {dir_str}")

                # Reseteamos
                estacionado_activo = False
                est_inicio = None
                est_fin = None
                est_coords = None

        # -----------------------------------------------------
        # LÓGICA 3: VEHÍCULO ENCENDIDO (Solo el primero)
        # -----------------------------------------------------
        # Solo reportamos si NO está agrupado en un estacionamiento (aunque tu regla dice solo el 1ro cronologico)
        if "ENCENDIDO" in raw_evento and not ya_reporto_encendido:
            if dentro_bodega:
                lug = "Bodega Monte Carlo"
            else:
                lug = obtener_direccion(lat, lon)

            reporte.append(f"-{hora_fmt} Se enciende en {lug}")
            ya_reporto_encendido = True

        # -----------------------------------------------------
        # LÓGICA 4: REPORTE POR TIEMPO / MOVIMIENTO
        # -----------------------------------------------------
        # Tu regla: "documenta calculando la ultima vez en movimiento"
        # Simplificación: Si sale reporte por tiempo y NO estamos estacionados, es tránsito.
        if "REPORTE POR TIEMPO" in raw_evento and not estacionado_activo:
             # Opcional: Solo reportar si han pasado X minutos desde el último reporte
             # Para no saturar, pondremos solo la ubicación actual
             pass
             # Nota: Lo he dejado en 'pass' (ignorar) porque en tus ejemplos de salida
             # NO aparecen reportes intermedios de "Reporte por tiempo" a menos que sea estacionado.
             # Si quieres que aparezca cada "ping" de movimiento, descomenta la linea de abajo:
             # reporte.append(f"-{hora_fmt} En tránsito por {obtener_direccion(lat, lon)}")

        # -----------------------------------------------------
        # LÓGICA 5: ENTRADA/SALIDA CIUDAD (Excluyendo Bodega)
        # -----------------------------------------------------
        if ("ENTRADA" in raw_evento or "SALIDA" in raw_evento) and "BODEGA" not in raw_evento:
            # Lógica de buffer simple: Registrarlo.
            # (La validación estricta de 5 min requiere mirar al futuro, aquí lo registramos)
            reporte.append(f"-{hora_fmt} {raw_evento.title()}")

    # --- CIERRE FINAL ---
    # Si el archivo termina y el carro seguía estacionado, cerramos el evento
    if estacionado_activo:
        dir_str = "Bodega Monte Carlo" if dentro_bodega else obtener_direccion(*est_coords)
        h_ini = est_inicio.strftime('%I:%M')
        h_fin = est_fin.strftime('%I:%M %p').lower().replace("am", "a.m.").replace("pm", "p.m.")
        reporte.append(f"-{h_ini} / {h_fin} Vehículo estacionado en {dir_str}")

    print("\n\n✨ Generación completada.")

    # Unir todo en un solo string (Argumentar)
    texto_final = ", ".join(reporte) + "."
    return texto_final

# --- EJECUCIÓN ---
resultado = generar_reporte_narrativo()

print("\n--- REPORTE FINAL ---")
print(resultado)

# Opcional: Guardar en archivo
with open("reporte_final.txt", "w") as f:
    f.write(resultado)

🔍 Buscando archivo...
📂 Procesando: /content/sample_data/reporte-NCX43H-15-01-2026-10-06-12.xlsx
⏳ Analizando eventos y coordenadas (esto toma un momento)...
Procesando fila 240 de 242...

✨ Generación completada.

--- REPORTE FINAL ---
.
